This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2021-11-15 12:33:39--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2021-11-15 12:33:39--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7618b693d86e06e9851c9fd0ba.dl.dropboxusercontent.com/cd/0/inline/BaAfNTZQlhOHvmO_jrIdTJ_CLWkL_QeHriceobf9Ubwn6W798y1M8R7q0mmAIP8yF2riyiGKa3tPKxUQdFzBKM2RLDeT1Lp_3Zje1oWe79NODcUx_Erw098TYtirR3dkW6LA3Eehj0dNi-cXWEBNvB2E/file# [following]
--2021-11-15 12:33:40--  https://uc7618b693d86e06e9851c9fd0ba.dl.dropboxusercontent.com/cd/0/inline/BaAfNTZQlhOHvmO_jrIdTJ_CLWkL_QeHriceobf9Ubwn6W798y1M8R7q0mmAIP8yF2riyiGKa3tPKxUQdFzBKM2RLDeT1Lp_3Zje1oWe79N

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [57]:
import pandas as pd

N = 15000
df = pd.read_csv('data/data_000637.txt', nrows=N)
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9
14996,1,1,17,3869200316,3400,15
14997,1,1,10,3869200316,3530,16
14998,1,1,8,3869200316,3533,18


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [58]:
# Each TDC count corresponds to 25/30 ns
# a unit of BX_COUNTER corresponds to 25 ns
# the ORBIT_CNT is increased every 'x' BX_COUNTER

# TDC_MEAS = 25/30 ns
# 1 BX_COUNTER = 25 ns
# ORBIT_CNT = x * BY_COUNTER (x is the max number of BY_COUNTER, from then on you have to increase BX_COUNTER)

max_bx_counter = df['BX_COUNTER'].max()
print(f'The number x is: {max_bx_counter}')

The number x is: 3563


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [59]:
df['ABS_TIME'] = df['TDC_MEAS'] * 25/30 + df['BX_COUNTER'] * 25 + df['ORBIT_CNT'] * 25 * max_bx_counter

duration_ns = df.iloc[-1]['ABS_TIME'] - df.iloc[0]['ABS_TIME']
duration_s = duration_ns / 10**9

print(f'Number of seconds: {duration_s}')

Number of seconds: 0.0133012783125


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [60]:
# Already calculated before (in point 3.)
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
0,1,0,123,3869200167,2374,26,3.446490e+14
1,1,0,124,3869200167,2374,27,3.446490e+14
2,1,0,63,3869200167,2553,28,3.446490e+14
3,1,0,64,3869200167,2558,19,3.446490e+14
4,1,0,64,3869200167,2760,25,3.446490e+14
...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,3.446490e+14
14996,1,1,17,3869200316,3400,15,3.446490e+14
14997,1,1,10,3869200316,3530,16,3.446490e+14
14998,1,1,8,3869200316,3533,18,3.446490e+14


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [56]:
df.groupby('TDC_CHANNEL').size().sort_values(ascending=False)[:3]

TDC_CHANNEL
139    108059
64      66020
63      64642
dtype: int64

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [68]:
# BOH! I don't get what you want me to do
not_empty = df['ORBIT_CNT'].unique()
print(not_empty.shape[0])

df_139 = df[df['TDC_CHANNEL'] == 139]
print(df_139[].shape[0])

150


AttributeError: 'DataFrame' object has no attribute 'unique'

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel